## Exercise - Partitioned Tables

Let us take care of this exercise related to partitioning to self evaluate our comfort level in working with partitioned tables.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - DML and Partitioning").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@762dd5e8


org.apache.spark.sql.SparkSession@762dd5e8

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Duration: **30 Minutes**
* Use data from **/data/nyse_all/nyse_data**
* Use database **YOUR_OS_USER_NAME_nyse**
* Create partitioned table **nyse_eod_part**
* Field Names: stockticker, tradedate, openprice, highprice, lowprice, closeprice, volume
* Determine correct data types based on the values
* Create Managed table with "," as delimiter.
* Partition Field should be **tradeyear** and of type **INT** (one partition for corresponding year)
* Insert data into partitioned table using dynamic partition mode.
* Here are the steps to come up with the solution.
  * Review the files under **/data/nyse_all/nyse_data** - determine data types (For example: tradedate should be INT and volume should be BIGINT)
  * Create database **YOUR_OS_USER_NAME_nyse** (if it does not exists)
  * Create non partitioned stage table
  * Load data into non partitioned stage table
  * Validate the count and also see that data is as expected by running simple select query.
  * Create partitioned table
  * Set required properties to use dynamic partition
  * Insert data into partitioned table - here is how you can compute year from tradedate of type int `year(to_date(cast(tradedate AS STRING), 'yyyyMMdd')) AS tradeyear`
  * Run below validate commands to validate

In [3]:
%%sql
USE itv002461_nyse 

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql
DROP TABLE IF EXISTS nyse_eod_part

++
||
++
++



In [9]:
%%sql
CREATE TABLE IF NOT EXISTS nyse_eod_part(
    stockticker STRING,
    tradedate INT,
    openprice FLOAT,
    highprice FLOAT,
    lowprice FLOAT,
    closeprice FLOAT,
    volume BIGINT  
)PARTITIONED BY (tradeyear INT)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [4]:
%%sql
CREATE TABLE IF NOT EXISTS nyse_eod_stage(
    stockticker STRING,
    tradedate INT,
    openprice FLOAT,
    highprice FLOAT,
    lowprice FLOAT,
    closeprice FLOAT,
    volume BIGINT  
)ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [5]:
%%sql
LOAD DATA LOCAL INPATH '/home/itv002461/tables' OVERWRITE INTO TABLE nyse_eod_stage

++
||
++
++



In [6]:
%%sql
SELECT count(1) FROM nyse_eod_stage 

+--------+
|count(1)|
+--------+
| 9384739|
+--------+



In [8]:
%%sql

SET hive.exec.dynamic.partition=true

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|hive.exec.dynamic...| true|
+--------------------+-----+



In [9]:
%%sql

SET hive.exec.dynamic.partition.mode=nonstrict

+--------------------+---------+
|                 key|    value|
+--------------------+---------+
|hive.exec.dynamic...|nonstrict|
+--------------------+---------+



In [10]:
%%sql
INSERT INTO TABLE nyse_eod_part PARTITION (tradeyear)
SELECT ns.*,year(to_date(cast(tradedate AS STRING), 'yyyyMMdd')) AS tradeyear
FROM nyse_eod_stage as ns

++
||
++
++



In [11]:
%%sql
SELECT COUNT(1) FROM nyse_eod_part

+--------+
|count(1)|
+--------+
| 9384739|
+--------+



In [33]:
%%sql
select tradeyear,count(*) as count 
FROM nyse_eod_part 
group by(tradeyear)

+---------+------+
|tradeyear| count|
+---------+------+
|     2003|358650|
|     2007|452691|
|     2015|759147|
|     2006|429757|
|     2013|638134|
|     1997|278396|
|     2014|702632|
|     2004|382896|
|     1998|294400|
|     2012|569725|
+---------+------+
only showing top 10 rows



In [30]:
%%sql
select sum(q.count) 
from(
        select count(*) as count 
        FROM nyse_eod_part 
        group by(tradeyear)
    ) as q

+----------+
|sum(count)|
+----------+
|   9384739|
+----------+



### Validation
Here are the instructions to validate the results.
* Run `hdfs dfs -ls /user/YOUR_OS_USER_NAME/warehouse/YOUR_OS_USER_NAME_nyse.db/nyse_eod_part`
* Run `SHOW PARTITIONS YOUR_OS_USER_NAME_nyse.nyse_eod_part`. You should see partitions for all the years using which you have loaded the data.
* Run `SELECT count(1) FROM YOUR_OS_USER_NAME_nyse.nyse_eod_part`. The count should match the number of records in our dataset.
* You can compare with the output generated by this simple Python code which is validated in our labs.

```
import pandas as pd
import glob

path = r'/data/nyse_all/nyse_data' # use your path
all_files = glob.glob(path + "/*.txt.gz")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.shape
```

In [31]:
import sys.process._
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [34]:
s"hdfs dfs -ls /user/${username}/warehouse/${username}_nyse.db/nyse_eod_part" !

Found 21 items
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=1997
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=1998
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=1999
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=2000
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=2001
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=2002
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 13:45 /user/itv002461/warehouse/itv002461_nyse.db/nyse_eod_part/tradeyear=2003
drwxr-xr-x   - itv002461 s

0

In [36]:
%%sql
SHOW PARTITIONS itv002461_nyse.nyse_eod_part

lastException: Throwable = null


+--------------+
|     partition|
+--------------+
|tradeyear=1997|
|tradeyear=1998|
|tradeyear=1999|
|tradeyear=2000|
|tradeyear=2001|
|tradeyear=2002|
|tradeyear=2003|
|tradeyear=2004|
|tradeyear=2005|
|tradeyear=2006|
+--------------+
only showing top 10 rows



In [38]:
%%sql 
SELECT count(1) FROM itv002461_nyse.nyse_eod_part

+--------+
|count(1)|
+--------+
| 9384739|
+--------+



In [7]:
import pandas as pd
import glob

path = r'/data/nyse_all/nyse_data' # use your path
all_files = glob.glob(path + "/*.txt.gz")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True)
frame.shape

(9384739, 7)